In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the SpaceX data into a pandas DataFrame
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a Dash application
app = dash.Dash(__name__)

# Get unique launch site names from the DataFrame
launch_sites = spacex_df['Launch Site'].unique().tolist()
dropdown_options = [{'label': 'All Sites', 'value': 'ALL'}] + \
                   [{'label': site, 'value': site} for site in launch_sites]

# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    dcc.Dropdown(id='site-dropdown',
                 options=dropdown_options,
                 value='ALL',
                 placeholder="Select a Launch Site here",
                 searchable=True),
    html.Br(),
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),
    html.P("Payload range (Kg):"),
    
    dcc.RangeSlider(
        id='payload-slider',
        min=0,  # Slider starting point
        max=10000,  # Slider ending point
        step=1000,  # Slider interval
        marks={i: '{}'.format(i) for i in range(0, 10001, 1000)},  # Marks on the slider
        value=[min_payload, max_payload]  # Current selected range
    ),
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# TASK 2: Callback for the pie chart based on dropdown
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value')]
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Count launches for each site when 'ALL' is selected
        site_counts = spacex_df['Launch Site'].value_counts().reset_index()
        site_counts.columns = ['Launch Site', 'count']
        title = "Launch Distribution for All Sites"
        fig = px.pie(site_counts, names='Launch Site', values='count', title=title)
    else:
        # Filter the DataFrame based on the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        class_counts = filtered_df['class'].value_counts().reset_index()
        class_counts.columns = ['class', 'count']
        class_counts['class'] = class_counts['class'].replace({1: 'Success', 0: 'Failure'})

        # Create a pie chart for success and failure counts for the selected site
        title = f"Launch Success and Failure for site {selected_site}"
        fig = px.pie(class_counts, names='class', values='count', title=title)

    return fig





# TASK 4: Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
# Define this callback function here
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_scatter_plot(selected_site, payload_range):
    low, high = payload_range
    if selected_site == 'ALL':
        # Filter based on payload range
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & 
                                (spacex_df['Payload Mass (kg)'] <= high)]
        title = "Payload vs. Outcome for All Sites"
    else:
        # Filter based on both site and payload range
        filtered_df = spacex_df[(spacex_df['Launch Site'] == selected_site) & 
                                (spacex_df['Payload Mass (kg)'] >= low) & 
                                (spacex_df['Payload Mass (kg)'] <= high)]
        title = f"Payload vs. Outcome for {selected_site}"

    # Create the scatter plot
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                     color='Booster Version Category', title=title)

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(port=8080)


C:\Users\ASUS\AppData\Local\Temp/ipykernel_16604/1526716457.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\ASUS\AppData\Local\Temp/ipykernel_16604/1526716457.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


Dash is running on http://127.0.0.1:8080/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jan/2024 14:18:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2024 14:18:54] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_4_1m1652870796.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2024 14:18:54] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_4_1m1652870796.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2024 14:18:54] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_4_0m1652870796.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2024 14:18:54] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_4_1m1652870796.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2024 14:18:54] "GET /_dash-component-suites/dash/deps/react@16.v2_4_1m1652870796.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2024 14:18:54] "GET /_dash-component-suites/dash/dash_table/bundle.v5_1_2m1652870796.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2024 14:18:54] "GET /_dash-component-suites/d

In [ ]:
pip install dash

In [ ]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"